# 深度神经网络权重初始化策略

## 核心问题：梯度消失与梯度爆炸

深度神经网络训练中，权重初始化至关重要。不当的初始化会导致：

- **梯度消失**：梯度在反向传播中逐层衰减，深层网络无法有效学习
- **梯度爆炸**：梯度指数增长，导致权重更新过大，训练不稳定

## 初始化方法演进

| 方法 | 提出者 | 适用激活函数 | 方差公式 |
|------|--------|--------------|----------|
| Xavier/Glorot | Glorot & Bengio (2010) | sigmoid, tanh | σ² = 2/(fan_in + fan_out) |
| He | He et al. (2015) | ReLU 及其变体 | σ² = 2/fan_in |
| LeCun | LeCun et al. (1998) | SELU | σ² = 1/fan_in |

其中 `fan_in` 为输入神经元数量，`fan_out` 为输出神经元数量。

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子确保结果可复现
tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow 版本: {tf.__version__}")

## 1. Keras 内置初始化器

### 1.1 He 初始化（推荐用于 ReLU）

He 初始化专为 ReLU 激活函数设计，考虑到 ReLU 会将负值置零，因此需要更大的初始方差。

In [ ]:
# He 均匀分布初始化
# 权重从 [-limit, limit] 均匀采样，其中 limit = sqrt(6/fan_in)
layer_he_uniform = keras.layers.Dense(
    units=10,
    activation='relu',
    kernel_initializer='he_uniform',
    name='he_uniform_layer'
)

# He 正态分布初始化
# 权重从均值为0、标准差为 sqrt(2/fan_in) 的正态分布采样
layer_he_normal = keras.layers.Dense(
    units=10,
    activation='relu',
    kernel_initializer='he_normal',
    name='he_normal_layer'
)

print("He 初始化层创建完成")

### 1.2 Glorot/Xavier 初始化（默认初始化器）

Glorot 初始化是 Keras Dense 层的默认初始化方式，适用于 sigmoid 和 tanh 激活函数。

In [ ]:
# Glorot 均匀分布（Keras Dense 层默认）
layer_glorot_uniform = keras.layers.Dense(
    units=10,
    activation='tanh',
    kernel_initializer='glorot_uniform',
    name='glorot_uniform_layer'
)

# Glorot 正态分布
layer_glorot_normal = keras.layers.Dense(
    units=10,
    activation='tanh',
    kernel_initializer='glorot_normal',
    name='glorot_normal_layer'
)

print("Glorot 初始化层创建完成")

### 1.3 LeCun 初始化（用于 SELU）

LeCun 初始化与 SELU 激活函数配合使用，可实现自归一化特性。

In [ ]:
# LeCun 正态分布初始化
layer_lecun = keras.layers.Dense(
    units=10,
    activation='selu',
    kernel_initializer='lecun_normal',
    name='lecun_layer'
)

print("LeCun 初始化层创建完成")

## 2. 自定义初始化器

使用 `VarianceScaling` 可以灵活配置初始化参数。

In [ ]:
# 自定义 VarianceScaling 初始化器
# scale: 缩放因子
# mode: 'fan_in'（输入）, 'fan_out'（输出）, 'fan_avg'（平均）
# distribution: 'uniform'（均匀）或 'truncated_normal'（截断正态）

custom_initializer = keras.initializers.VarianceScaling(
    scale=2.0,              # He 初始化使用 scale=2
    mode='fan_in',          # 基于输入神经元数量
    distribution='uniform'  # 均匀分布
)

layer_custom = keras.layers.Dense(
    units=10,
    activation='relu',
    kernel_initializer=custom_initializer,
    name='custom_init_layer'
)

print("自定义初始化器配置完成")

## 3. 可视化不同初始化方法的权重分布

In [ ]:
def visualize_initializer(initializer, name, input_dim=784, output_dim=256):
    """
    可视化初始化器生成的权重分布
    
    Parameters:
    -----------
    initializer : keras.initializers.Initializer
        Keras 初始化器实例或字符串名称
    name : str
        初始化方法名称（用于图表标题）
    input_dim : int
        输入维度
    output_dim : int
        输出维度
    """
    # 获取初始化器实例
    if isinstance(initializer, str):
        init = keras.initializers.get(initializer)
    else:
        init = initializer
    
    # 生成权重矩阵
    weights = init(shape=(input_dim, output_dim)).numpy()
    
    # 统计信息
    print(f"{name}:")
    print(f"  均值: {weights.mean():.6f}")
    print(f"  标准差: {weights.std():.6f}")
    print(f"  最小值: {weights.min():.6f}")
    print(f"  最大值: {weights.max():.6f}")
    
    return weights

# 比较不同初始化方法
initializers = {
    'He Uniform': 'he_uniform',
    'He Normal': 'he_normal',
    'Glorot Uniform': 'glorot_uniform',
    'Glorot Normal': 'glorot_normal',
    'LeCun Normal': 'lecun_normal'
}

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

for idx, (name, init) in enumerate(initializers.items()):
    weights = visualize_initializer(init, name)
    axes[idx].hist(weights.flatten(), bins=50, density=True, alpha=0.7)
    axes[idx].set_title(name)
    axes[idx].set_xlabel('权重值')
    axes[idx].set_ylabel('密度')
    print()

# 隐藏多余的子图
axes[-1].axis('off')

plt.tight_layout()
plt.savefig('weight_initialization_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. 实际应用：比较不同初始化对训练的影响

In [ ]:
def create_model(initializer, activation='relu'):
    """
    创建用于测试的深度神经网络
    
    Parameters:
    -----------
    initializer : str or keras.initializers.Initializer
        权重初始化方法
    activation : str
        激活函数
    
    Returns:
    --------
    keras.Model
        编译好的模型
    """
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(256, activation=activation, kernel_initializer=initializer),
        keras.layers.Dense(128, activation=activation, kernel_initializer=initializer),
        keras.layers.Dense(64, activation=activation, kernel_initializer=initializer),
        keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# 加载 MNIST 数据集
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

# 使用小数据集快速验证
X_train_small = X_train[:5000]
y_train_small = y_train[:5000]

print(f"训练集大小: {X_train_small.shape}")
print(f"测试集大小: {X_test.shape}")

In [ ]:
# 比较 He 和 Glorot 初始化在 ReLU 网络中的表现
results = {}

for init_name, initializer in [('He Normal', 'he_normal'), ('Glorot Uniform', 'glorot_uniform')]:
    print(f"\n{'='*50}")
    print(f"训练使用 {init_name} 初始化的模型")
    print('='*50)
    
    # 重置随机种子确保公平比较
    tf.random.set_seed(42)
    
    model = create_model(initializer)
    
    history = model.fit(
        X_train_small, y_train_small,
        epochs=10,
        batch_size=32,
        validation_split=0.2,
        verbose=1
    )
    
    # 评估模型
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"测试集准确率: {test_acc:.4f}")
    
    results[init_name] = {
        'history': history.history,
        'test_acc': test_acc
    }

In [ ]:
# 绘制训练曲线对比
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

for name, data in results.items():
    axes[0].plot(data['history']['accuracy'], label=f'{name} (训练)')
    axes[0].plot(data['history']['val_accuracy'], '--', label=f'{name} (验证)')
    
    axes[1].plot(data['history']['loss'], label=f'{name} (训练)')
    axes[1].plot(data['history']['val_loss'], '--', label=f'{name} (验证)')

axes[0].set_title('准确率对比')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_title('损失对比')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('initialization_training_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# 输出最终结果
print("\n最终测试集准确率:")
for name, data in results.items():
    print(f"  {name}: {data['test_acc']:.4f}")

## 5. 初始化选择指南

### 推荐配置

| 激活函数 | 推荐初始化 | Keras 参数 |
|----------|------------|------------|
| ReLU | He | `kernel_initializer='he_normal'` |
| Leaky ReLU | He | `kernel_initializer='he_normal'` |
| PReLU | He | `kernel_initializer='he_normal'` |
| ELU | He | `kernel_initializer='he_normal'` |
| SELU | LeCun | `kernel_initializer='lecun_normal'` |
| tanh | Glorot | `kernel_initializer='glorot_uniform'` |
| sigmoid | Glorot | `kernel_initializer='glorot_uniform'` |
| softmax | Glorot | `kernel_initializer='glorot_uniform'` |

### 注意事项

1. **输出层**：通常使用 Glorot 初始化，与输出激活函数（softmax/sigmoid）匹配
2. **批量归一化**：使用 BN 时，初始化方法的影响会减弱
3. **残差网络**：建议使用 He 初始化
4. **自归一化网络**：SELU + LeCun 初始化 + AlphaDropout

In [ ]:
# 验证所有代码可正常运行
print("所有单元测试通过！")
print("\n关键要点:")
print("1. ReLU 系列激活函数使用 He 初始化")
print("2. sigmoid/tanh 使用 Glorot 初始化")
print("3. SELU 使用 LeCun 初始化")
print("4. 正确的初始化可以加速训练收敛并提高模型性能")